# Figures associated to fibroblast processing

**You need to tun the notebook 2_ first!!!!**

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk
import itertools
import matplotlib.pylab as pylab
from datetime import date

In [ ]:
from functions import adata_plot_KOvsWT_2bars, stat_annot_gene, plot_WT_KO_genes
from functions import dict_WT_KO_colors

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
params = {'figure.dpi': 200,
          'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large', 
         'axes.grid': False}
pylab.rcParams.update(params)
# mpl.style.use('default')

pd.set_option('display.max_columns', None)

In [ ]:
os.makedirs('results/figures/F2', exist_ok=True)

## Functions and variables

In [ ]:
%store -r seed

## Load adata

In [ ]:
adata_all_harmony = sc.read_h5ad('adatas/adata_all_harmony.h5')
adata_fb = adata_all_harmony[adata_all_harmony.obs['merged_cell_type_name'] == 'FB']
adata_fb.uns['batch_colors'] = adata_all_harmony.uns['batch_colors']

In [ ]:
sc.pp.filter_genes(adata_fb, min_counts=1)

In [ ]:
sc.pp.pca(adata_fb, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_fb, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_fb, random_state=seed, n_neighbors=int(len(adata_fb) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_fb, use_raw=False)

sc.pp.pca(adata_fb, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_fb, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_fb, random_state=seed, n_neighbors=int(len(adata_fb) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')

We see that some populations (*Coch*$^+$ or *Thbs4*$^+$) are separated into two populations. We are going to separate them by finding the corresponding DEGs.

In [ ]:
dict_fbs = {    '0: FB Pdpn$^+$Il1rl1$^+$': ['Pdpn', 'Sod2', 'Il1rl1', 'Mif', 'Twist2', 'Pgk1', 'Gm29408', 'Ptges', 'Ctsk', 'Pgam1', 'Ets2', 
                                             'Eno1', 'Znrf1', 'Sh3pxd2b', 'Tnfaip2', 'Olfr1033', 'B2m', 'Fosl1', 'Plin2', 'Dlg1', 'Col7a1',], 
                '1: FB Msc$^+$Itga7$^+$': ['Bok', 'Lrrc15', 'Msc', 'Itga7', 'Etl4', 'Mcam', 'Ednrb', 'Blnk', 'Mrln', 'Myf5', 'Popdc3', 'Acta2', 
                                           'Raspn', 'Tnik', 'Sytl2', 'Zic1', 'Tnnt1', 'Jsrp1', 'Vgll2', 'Ank1', 'Cobl',], 
                '2: FB Serpine2$^+$Col23a1$^+$': ['Serpine2', 'Shox2', 'Wif1', 'Col23a1', 'Gm48159', 'Cd24a', 'Nav2', 'Tfap2c', 'Tanc1', 'Scube3', 
                                                  'Cblb', 'Fgfr2', 'Olfm', 'Prdm1', 'Alx3', 'Inhba', 'Dio3', 'Rspo1'], 
                '3: FB Cxcl1$^+$Ccl2$^+$': ['Cxcl1', 'Ccl7', 'Has1', 'Cxcl2', 'Tnfaip6', 'Cxcl10', 'Ccl2', 'Gfpt2', 'Il6', 'Gpc3', 'Ccl11', 'Clec2d', 
                                            'Slc43a3', 'Pkdcc', 'C1s1', 'Pvr', 'Cd44', 'Mt1', 'Tnfaip6', 'Mt2', 'Abi3bp', 'Has1',],
                '4: FB Clec3b$^+$Ly6a$^+$': ['Clec3b', 'Tnxb', 'Ly6a', 'Cd248', 'Ecm1', 'Tppp3', 'Islr', 'Ltbp4', 'Ly6c1', 'Nfib', 'Itm2a', 'Fbn1', 
                                             'Igf1', 'Lysmd2', 'Ebf1', 'Adamtsl4', 'C1s1', 'Igfbp6', 'Cxcl12', 'Mgst3', 'Cygb', 'Sema3c', ],
                '5: FB Rab37$^+$Col22a1$^+$': ['Rab37', 'Col22a1', 'F13a1', 'Htra4', 'Tspan15', 'Rgcc', 'Hbegf', 'Cd55', 'Capg', 'Itga6', 'Pcsk6', 'Sema3c', 'Htra4', 'Sox5', 'Rgs5', 'Aldh1a3', 'Sema3d', 
                                         'Cmtm8', 'Ackr2'], 
                '6: FB Cilp2$^+$Acan$^+$': ['Cilp2', 'Chad', 'Egfl6', 'Comp', 'Fmod', 'Tnmd', 'Ecrg4', 'Kera', 'Acan', 'Scx', 'Fibin', 'Tspan2'], 
                '7: FB Lox$^+$Adam33$^+$': ['Mfap4', 'Lox', 'Adam33', 'Eln', 'Pdgfrl', 'Ccn5', 'Fbln5', 'Vgll3', 'Wnt5a', 'Osr2', 'Igfbp2', 'Phldb2', 
                                            'Pth1r', 'Ptk7', 'Col8a1', 'Slit2', 'Mmp16', 'C1qtnf3', 'Lrrc15'],
                '8: FB Coch$^+$Emid1$^+$': ['Coch', 'Emid1', 'Gldn', 'Ntn5', 'Tnmd', 'Col16a1', 'Col11a1',  'Crabp1', 'Mafb', 'Fmod', 'Fbn2', 
                                            'Cxcr4', 'Ogn', 'Kera', 'Wfdc1', 'Hs3st6', 'Pi16', 'Trib2'],     
                '9: FB Cfh$^+$Spp1$^+$': ['Cfh', 'Alpl', 'Lifr', 'Sp7', 'Spp1', 'Runx2', 'Cp', 'Mmp13', 'Ptprd', 'Hp',  'Ibsp', 'Cadm1', 'Lgmn', 
                                           'Limch1', 'Vdr', 'Dapk2', 'Dlx5', 'Sept4'],
                  }

In [ ]:
dict_fbs_colors = {  '0: FB Pdpn$^+$Il1rl1$^+$': '#67dcf1',
                     '1: FB Msc$^+$Itga7$^+$': '#df7935',
                     '2: FB Serpine2$^+$Col23a1$^+$': '#20668d',
                     '3: FB Cxcl1$^+$Ccl2$^+$': '#a05a2c',
                     '4: FB Clec3b$^+$Ly6a$^+$': '#ff5599',
                     '5: FB Rab37$^+$Col22a1$^+$': '#d40055',
                     '6: FB Cilp2$^+$Acan$^+$': '#8e0c6f',
                     '7: FB Lox$^+$Adam33$^+$': '#d3bc5f',
                     '8: FB Coch$^+$Emid1$^+$': '#4d0765',
                     '9: FB Cfh$^+$Spp1$^+$': '#229eb2',
                     }

dict_fbs_colors_nan = {  '0: FB Pdpn$^+$Il1rl1$^+$': '#67dcf1',
                         '1: FB Msc$^+$Itga7$^+$': '#df7935',
                         '2: FB Serpine2$^+$Col23a1$^+$': '#20668d',
                         '3: FB Cxcl1$^+$Ccl2$^+$': '#a05a2c',
                         '4: FB Clec3b$^+$Ly6a$^+$': '#ff5599',
                         '5: FB Rab37$^+$Col22a1$^+$': '#d40055',
                         }

In [ ]:
sc.tl.leiden(adata_fb, resolution=4)
assign_cats(adata_fb, dict_cats=dict_fbs, min_score=0.4, quantile_gene_sel=0.7, key_added='subtype')


adata_fb.obs['subtype_name'] = [i.split(': ')[1] for i in adata_fb.obs['subtype']]
adata_fb.obs['subtype_name'] = adata_fb.obs['subtype_name'].astype('category')
adata_fb.obs['subtype_name'].cat.reorder_categories([i.split(': ')[1] for i in dict_fbs.keys()], inplace=True)

adata_fb.obs['subtype_number'] = [i.split(': ')[0] for i in adata_fb.obs['subtype']]
adata_fb.obs['subtype_number'] = adata_fb.obs['subtype_number'].astype('category')
adata_fb.obs['subtype_number'].cat.reorder_categories([i.split(': ')[0] for i in dict_fbs.keys()], inplace=True)


adata_fb.uns['subtype_colors'] = list(dict_fbs_colors.values())
adata_fb.uns['subtype_name_colors'] = list(dict_fbs_colors.values())
adata_fb.uns['subtype_number_colors'] = list(dict_fbs_colors.values())

In [ ]:
sc.tl.umap(adata_fb, min_dist=0.35)

## UMAP of fibroblast subpopulations and batch

#### Supp Fig to Fig 3

In [ ]:
adata_fb_WT = adata_fb[adata_fb.obs['batch'].isin(['WT1', 'WT2'])]
adata_fb_WT.uns['batch_colors'] = [dict_WT_KO_colors['WT1'], dict_WT_KO_colors['WT2'], ]
adata_fb_KO = adata_fb[adata_fb.obs['batch'].isin(['KO1', 'KO2'])]
adata_fb_KO.uns['batch_colors'] = [dict_WT_KO_colors['KO1'], dict_WT_KO_colors['KO2'], ]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 4))
s = 18

sc.pl.umap(adata_fb, color='subtype_number', s=s, ax = axs, 
           legend_loc='on data', frameon=False, show=False, 
           legend_fontoutline=4, legend_fontsize=14)
sc.pl.umap(adata_fb, color='subtype', s=s, ax = axs, 
           frameon=False, show=False)

title_obj = axs.set_title('') 

plt.tight_layout()

plt.savefig(f'results/figures/F2/makers_FB_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
plt.savefig(f'results/figures/F2/Supp_Fig3A_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 7))
s = 18


sc.pl.umap(adata_fb_WT, color='batch', s=s, ax = axs.ravel()[0], 
           frameon=False, show=False, na_in_legend=False)


sc.pl.umap(adata_fb_WT, color='subtype_number', s=s, ax=axs.ravel()[1], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='condition')
sc.pl.umap(adata_fb_WT, color='subtype', s=s, ax = axs.ravel()[1], 
           frameon=False, show=False, na_in_legend=False, title='FB type')



sc.pl.umap(adata_fb_KO, color='batch', s=s, ax = axs.ravel()[2], 
           frameon=False, show=False, na_in_legend=False, title='')


sc.pl.umap(adata_fb_KO, color='subtype_number', s=s, ax=axs.ravel()[3], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='')


plt.tight_layout()

plt.savefig(f'results/figures/F2/makers_FB_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
plt.savefig(f'results/figures/F2/Supp_Fig3A_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

#### Fig 3A

In [ ]:
selected_cell_types = list(dict_fbs_colors_nan.keys())

In [ ]:
selected_cell_types

In [ ]:
adata_fb.obs['subtype_nan'] = [i if i in selected_cell_types else np.NaN for i in adata_fb.obs['subtype']]
adata_fb.obs['subtype_nan'] = adata_fb.obs['subtype_nan'].astype('category')
adata_fb.obs['subtype_nan'].cat.reorder_categories(selected_cell_types, inplace=True)

adata_fb.obs['subtype_nan_name'] = [i.split(': ')[1] if i in selected_cell_types else np.NaN for i in adata_fb.obs['subtype_nan']]
adata_fb.obs['subtype_nan_name'] = adata_fb.obs['subtype_nan_name'].astype('category')
adata_fb.obs['subtype_nan_name'].cat.reorder_categories([i.split(': ')[1] for i in selected_cell_types], inplace=True)

adata_fb.obs['subtype_nan_number'] = [i.split(': ')[0] if i in selected_cell_types else np.NaN for i in adata_fb.obs['subtype_nan']]
adata_fb.obs['subtype_nan_number'] = adata_fb.obs['subtype_nan_number'].astype('category')
adata_fb.obs['subtype_nan_number'].cat.reorder_categories([i.split(': ')[0] for i in selected_cell_types], inplace=True)


adata_fb.uns['subtype_nan_colors'] = list(dict_fbs_colors_nan.values())
adata_fb.uns['subtype_nan_name_colors'] = list(dict_fbs_colors_nan.values())
adata_fb.uns['subtype_nan_number_colors'] = list(dict_fbs_colors_nan.values())

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 4))
s = 12

sc.pl.umap(adata_fb, color='subtype_nan_number', s=s, ax = axs, 
           legend_loc='on data', frameon=False, show=False, 
           legend_fontoutline=4, legend_fontsize=14, na_in_legend=False)
sc.pl.umap(adata_fb, color='subtype_nan', s=s, ax = axs, 
           frameon=False, show=False, na_in_legend=False)

title_obj = axs.set_title('') 

plt.tight_layout()

plt.savefig(f'results/figures/F2/makers_FB_nan_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
plt.savefig(f'results/figures/F2/Fig2A_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
# We repeat the variable naming here (it was done before) because we need to add nan vars to the new adata_fb_WT and adata_fb_KO
adata_fb_WT = adata_fb[adata_fb.obs['batch'].isin(['WT1', 'WT2'])]
adata_fb_WT.uns['subtype_nan_colors'] = adata_fb.uns['subtype_nan_colors']
adata_fb_WT.uns['subtype_nan_number_colors'] = adata_fb.uns['subtype_nan_number_colors']
adata_fb_WT.uns['batch_colors'] = [dict_WT_KO_colors['WT1'], dict_WT_KO_colors['WT2']]

adata_fb_KO = adata_fb[adata_fb.obs['batch'].isin(['KO1', 'KO2'])]
adata_fb_KO.uns['subtype_nan_colors'] = adata_fb.uns['subtype_nan_colors']
adata_fb_KO.uns['subtype_nan_number_colors'] = adata_fb.uns['subtype_nan_number_colors']
adata_fb_KO.uns['batch_colors'] = [dict_WT_KO_colors['KO1'], dict_WT_KO_colors['KO2']]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 7))
s = 18


sc.pl.umap(adata_fb_WT, color='batch', s=s, ax = axs.ravel()[0], 
           frameon=False, show=False, na_in_legend=False)


sc.pl.umap(adata_fb_WT, color='subtype_nan_number', s=s, ax=axs.ravel()[1], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='condition')
sc.pl.umap(adata_fb_WT, color='subtype_nan', s=s, ax = axs.ravel()[1], 
           frameon=False, show=False, na_in_legend=False, title='FB type')



sc.pl.umap(adata_fb_KO, color='batch', s=s, ax = axs.ravel()[2], 
           frameon=False, show=False, na_in_legend=False, title='')


sc.pl.umap(adata_fb_KO, color='subtype_nan_number', s=s, ax=axs.ravel()[3], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='')


plt.tight_layout()

plt.savefig(f'results/figures/F2/makers_FB_nan_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
plt.savefig(f'results/figures/F2/Fig2A_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

## Proportion of cell types within WT and KO

In [ ]:
adata_plot_KOvsWT_2bars(adata_fb, list(dict_fbs.keys()), do_return=False, col_cell_type='subtype', sort_by='diff', add_number_to_label=True,)
plt.savefig(f'results/figures/F2/barplot_fb_diff_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
adata_plot_KOvsWT_2bars(adata_fb, list(dict_fbs.keys()), do_return=False, col_cell_type='subtype', sort_by='counts', add_number_to_label=True)
plt.savefig(f'results/figures/F2/barplot_fb_counts_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
adata_plot_KOvsWT_2bars(adata_fb, list(dict_fbs.keys()), do_return=False, col_cell_type='subtype', sort_by='label', add_number_to_label=True)
plt.savefig(f'results/figures/F2/barplot_fb_label_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

## Table with markers and p-values

In [ ]:
sc.tl.rank_genes_groups(adata_fb, groupby='subtype')
names = pd.DataFrame(adata_fb.uns['rank_genes_groups']['names'])
pvals = pd.DataFrame(adata_fb.uns['rank_genes_groups']['pvals_adj'])

In [ ]:
for cell_type, list_cell_markers in dict_fbs.items():
    print(cell_type)
    sc.pl.umap(adata_fb, color=[i for i in list_cell_markers if i in adata_fb.var_names], alpha=0.5, ncols=5, cmap=magma, use_raw=False)

In [ ]:
# Create a DataFrame with 30 most important genes
list_dfs = []
for subtype, genes in dict_fbs.items():
    df = pd.DataFrame({f'{subtype} genes': names[subtype], f'p-value {subtype}': pvals[subtype]}, dtype=object)
    df_short = df[df[f'{subtype} genes'].isin(genes)].sort_values(by=f'p-value {subtype}', ascending=True).iloc[:30].reset_index(drop=True)
    df[f'p-value {subtype}'] = df[f'p-value {subtype}'].apply(lambda x: '%.3E' % x).values.tolist()
    list_dfs.append(df)

In [ ]:
df_all_celltypes = pd.concat(list_dfs, axis=1).iloc[:50]
df_all_celltypes.to_csv(f'results/{date.today().strftime("%Y_%m_%d")}-FB_DEGs.csv.csv', index=False)
df_all_celltypes

In [ ]:
dict_fbs = {    '0: FB Pdpn$^+$Il1rl1$^+$': ['Pdpn', 'Sod2', 'Il1rl1', 'Mif', 'Twist2', 'Pgk1', 'Gm29408', 'Ptges', 'Ctsk', 'Pgam1', 'Ets2', 
                                             'Eno1', 'Znrf1', 'Sh3pxd2b', 'Tnfaip2', 'Olfr1033', 'B2m', 'Fosl1', 'Plin2', 'Dlg1', 'Col7a1',], 
                '1: FB Msc$^+$Itga7$^+$': ['Bok', 'Lrrc15', 'Msc', 'Itga7', 'Etl4', 'Mcam', 'Ednrb', 'Blnk', 'Mrln', 'Myf5', 'Popdc3', 'Acta2', 
                                           'Raspn', 'Tnik', 'Sytl2', 'Zic1', 'Tnnt1', 'Jsrp1', 'Vgll2', 'Ank1', 'Cobl',], 
                '2: FB Serpine2$^+$Col23a1$^+$': ['Serpine2', 'Shox2', 'Wif1', 'Col23a1', 'Gm48159', 'Cd24a', 'Nav2', 'Tfap2c', 'Tanc1', 'Scube3', 
                                                  'Cblb', 'Fgfr2', 'Olfm', 'Prdm1', 'Alx3', 'Inhba', 'Dio3', 'Rspo1'], 
                '3: FB Cxcl1$^+$Ccl2$^+$': ['Cxcl1', 'Ccl7', 'Has1', 'Cxcl2', 'Tnfaip6', 'Cxcl10', 'Ccl2', 'Gfpt2', 'Il6', 'Gpc3', 'Ccl11', 'Clec2d', 
                                            'Slc43a3', 'Pkdcc', 'C1s1', 'Pvr', 'Cd44', 'Mt1', 'Tnfaip6', 'Mt2', 'Abi3bp', 'Has1',],
                '4: FB Clec3b$^+$Ly6a$^+$': ['Clec3b', 'Tnxb', 'Ly6a', 'Cd248', 'Ecm1', 'Tppp3', 'Islr', 'Ltbp4', 'Ly6c1', 'Nfib', 'Itm2a', 'Fbn1', 
                                             'Igf1', 'Lysmd2', 'Ebf1', 'Adamtsl4', 'C1s1', 'Igfbp6', 'Cxcl12', 'Mgst3', 'Cygb', 'Sema3c', ],
                '5: FB Rab37$^+$Col22a1$^+$': ['Rab37', 'Col22a1', 'F13a1', 'Htra4', 'Tspan15', 'Rgcc', 'Hbegf', 'Cd55', 'Capg', 'Itga6', 'Pcsk6', 'Sema3c', 'Htra4', 'Sox5', 'Rgs5', 'Aldh1a3', 'Sema3d', 
                                         'Cmtm8', 'Ackr2'], 
                '6: FB Cilp2$^+$Acan$^+$': ['Cilp2', 'Chad', 'Egfl6', 'Comp', 'Fmod', 'Tnmd', 'Ecrg4', 'Kera', 'Acan', 'Scx', 'Fibin', 'Tspan2'], 
                '7: FB Lox$^+$Adam33$^+$': ['Mfap4', 'Lox', 'Adam33', 'Eln', 'Pdgfrl', 'Ccn5', 'Fbln5', 'Vgll3', 'Wnt5a', 'Osr2', 'Igfbp2', 'Phldb2', 
                                            'Pth1r', 'Ptk7', 'Col8a1', 'Slit2', 'Mmp16', 'C1qtnf3', 'Lrrc15'],
                '8: FB Coch$^+$Emid1$^+$': ['Coch', 'Emid1', 'Gldn', 'Ntn5', 'Tnmd', 'Col16a1', 'Col11a1',  'Crabp1', 'Mafb', 'Fmod', 'Fbn2', 
                                            'Cxcr4', 'Ogn', 'Kera', 'Wfdc1', 'Hs3st6', 'Pi16', 'Trib2'],     
                '9: FB Cfh$^+$Spp1$^+$': ['Cfh', 'Alpl', 'Lifr', 'Sp7', 'Spp1', 'Runx2', 'Cp', 'Mmp13', 'Ptprd', 'Hp',  'Ibsp', 'Cadm1', 'Lgmn', 
                                           'Limch1', 'Vdr', 'Dapk2', 'Dlx5', 'Sept4'],
                  }

## Dotplot with markers of fibroblast subtypes

In [ ]:
markers_fb_dotplot = [
     'Pdpn', 'Il1rl1', 'Ptgs2', 'Hilpda', 
     'Msc', 'Itga7', 'Cdh15', 'Blnk', 
     'Serpine2', 'Shox2', 'Cd24a', 'Col23a1',  
     'Ccl2', 'Cxcl1', 'Ccl7', 'Cxcl2', 
     'Clec3b', 'Ly6a', 'Ltbp4', 'Cd248',  
     'Htra4', 'F13a1', 'Col22a1', 'Rab37',  
     'Cilp2', 'Acan', 'Chad', 'Egfl6', 
     'Igfbp2', 'Phldb2', 'Col8a1', 'Ccn5', 
     'Coch', 'Emid1', 'Gldn', 'Ntn5', 
     'Spp1', 'Cfh', 'Lifr', 'Alpl']

In [ ]:
sc.pl.dotplot(adata_fb, groupby='subtype', var_names=markers_fb_dotplot, dendrogram=False, show=False, 
             categories_order=list(dict_fbs.keys()), use_raw=False, vmax=3, smallest_dot=0.3)

plt.savefig(f'results/figures/F2/markers_dotplot_FB_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

## Heatmaps of relevant pathways

In [ ]:
complement_genes = ['C1qa', 'C1qb', 'C1qbp', 'C1qc', 'C1ra', 'C1rb', 'C1rl', 'C1s1', 'C1s2', 
                    'C2', 'C3', 'C4a', 'C4b', 'C8g', 'Cd46', 
                    'Cfb', 'Cfd', 'Cfh',  'Cfhr1', 'Cfhr2', 'Cfp', 'Clu', 'Colec10', 'Colec11', 
                    'Crp', 'F3', 'Fcna',  'Il1b', 'Krt1', 'Masp2', 
                    'Phb', 'Procr', 'Rgcc', 'Thbd', 'Trbc1', 'Trbc2', 'Trdc', 
                    'Trdv4', 'Trem2']  # GO:0006956 - filtering negative regulation of complement activation GO:0045916


complement_genes_filtered = ['C1qbp', 'C1s1', 'C3', 'C4b', 'Cfh', 'Clu', 'F3', 'Phb', 'Rgcc', 'Thbd']



cytokine_interaction_genes = ['Ackr2', 'Ackr3', 'Ackr4', 'Adam17', 'Amh', 'Angpt1', 'Angpt2', 'Angpt4', 'Babam2', 'Bdnf', 'Bex3', 'Bmp3', 'Bmp4', 'Bmp8a', 'Cadm4', 'Casp3', 'Casp8', 'Ccdc88a', 'Ccl11', 
                              'Ccl12', 'Ccl19', 'Ccl2', 'Ccl20', 'Ccl21a', 'Ccl24', 'Ccl25', 'Ccl27a', 'Ccl28', 'Ccl3', 'Ccl4', 'Ccl5', 'Ccl6', 'Ccl7', 'Ccl8', 'Ccl9', 'Ccr1', 'Ccr10', 'Ccr5', 'Ccr7', 
                              'Ccr9', 'Ccrl2', 'Cd2ap', 'Cd300lf', 'Cd4', 'Cd44', 'Cd70', 'Cd74', 'Cdh5', 'Ceacam1', 'Cflar', 'Cklf', 'Clcf1', 'Cnih4', 'Cntfr', 'Creb3', 'Crlf1', 'Crlf2', 'Csf1', 'Csf2', 
                              'Csf2ra', 'Csf2rb', 'Csf2rb2', 'Csf3', 'Csf3r', 'Ctf1', 'Ctf2', 'Cx3cl1', 'Cx3cr1', 'Cxcl1', 'Cxcl10', 'Cxcl11', 'Cxcl12', 'Cxcl13', 'Cxcl14', 'Cxcl15', 'Cxcl16', 'Cxcl2', 
                              'Cxcl3', 'Cxcl5', 'Cxcl9', 'Cxcr2', 'Cxcr4', 'Cxcr5', 'Cxcr6', 'Dab2ip', 'Ddt', 'Defb1', 'Defb6', 'Dnaja3', 'Ecm1', 'Eda', 'Edaradd', 'Efna5', 'Eng', 'Epor', 'Erap1', 'F3', 
                              'Fadd', 'Fem1b', 'Fermt2', 'Fkbp1a', 'Frs2', 'Fzd4', 'Gata3', 'Gdnf', 'Gfra2', 'Gfra4', 'Ghr', 'Gm13272', 'Gm13283', 'Gm13305', 'Gm21970', 'Gpr35', 'Gpr75', 'Grb2', 'Grem1', 
                              'Grin2b', 'Hsd17b7', 'Ifnar1', 'Ifnar2', 'Ifngr1', 'Ifngr2', 'Ifnlr1', 'Ifnz', 'Il10', 'Il10ra', 'Il10rb', 'Il11', 'Il11ra1', 'Il12a', 'Il12rb1', 'Il12rb2', 'Il13ra1', 'Il13ra2', 
                              'Il15', 'Il15ra', 'Il16', 'Il17f', 'Il17ra', 'Il17rb', 'Il17rc', 'Il17rd', 'Il17re', 'Il18', 'Il18r1', 'Il18rap', 'Il1b', 'Il1r1', 'Il1r2', 'Il1rap', 'Il1rl1', 'Il1rl2', 
                              'Il1rn', 'Il20ra', 'Il20rb', 'Il21r', 'Il23a', 'Il27ra', 'Il2ra', 'Il2rg', 'Il31ra', 'Il33', 'Il34', 'Il3ra', 'Il4ra', 'Il6', 'Il6ra', 'Il6st', 'Il7', 'Irak1', 'Irak4', 
                              'Itch', 'Itgb3', 'Jak1', 'Jak2', 'Jak3', 'Kitl', 'Lefty1', 'Lefty2', 'Lepr', 'Lif', 'Lifr', 'Lta', 'Ltb', 'Lyn', 'Mif', 'Mpl', 'Msmp', 'Myd88', 'Nars', 'Nes', 'Ngf', 'Ngfr', 
                              'Nol3', 'Nradd', 'Nsmaf', 'Ntf3', 'Ntf5', 'Ntrk1', 'Nucb2', 'Nup85', 'Osm', 'Osmr', 'Pdcl3', 'Pf4', 'Pgf', 'Pibf1', 'Pik3r1', 'Plcg1', 'Prdm4', 'Prl2c3', 'Prl2c5', 'Prl7d1', 
                              'Prlr', 'Ptpn6', 'Pycard', 'Rasl11b', 'Ripk1', 'Rnf41', 'S100a14', 'Sdcbp', 'Sh2b3', 'Shc1', 'Siva1', 'Smad2', 'Smad3', 'Smad6', 'Smad7', 'Smurf2', 'Snx25', 'Snx6', 'Socs2', 
                              'Spred1', 'Spred2', 'Spred3', 'Stap1', 'Stat1', 'Stat3', 'Syk', 'Tgfb1', 'Tgfb2', 'Tgfb3', 'Tgfbr1', 'Tgfbr2', 'Tgfbr3', 'Tgfbr3l', 'Tgfbrap1', 'Timm50', 'Tlr5', 'Tmbim1', 
                              'Tnf', 'Tnfsf10', 'Tnfsf11', 'Tnfsf12', 'Tnfsf13b', 'Tnfsf15', 'Tnfsf4', 'Tnfsf8', 'Tnfsf9', 'Tollip', 'Tradd', 'Traf1', 'Traf2', 'Traf3', 'Traf4', 'Traf5', 'Traf6', 'Trim37', 
                              'Trip6', 'Tslp', 'Tyk2', 'Usp15', 'Vegfa', 'Vegfb', 'Vegfc', 'Vegfd', 'Xcl1', 'Xcr1', 'Zfp110', 'Zfp369']  # GO:0005126 + GO:0004896

cytokine_interaction_genes_filtered = ['Ackr2', 'Adam17', 'Bex3', 'Bmp4', 'Casp3', 'Ccl11', 'Ccl19', 'Ccl2', 'Ccl5', 'Ccl7', 
                                       'Cd2ap', 'Cd44', 'Cflar', 'Cnih4', 'Crlf1', 'Csf1', 'Cxcl1', 'Cxcl10', 'Cxcl12',
                                       'Cxcl14', 'Cxcl2', 'Cxcl5', 'Dab2ip', 'Ddt', 'Ecm1', 'Eng', 'Fermt2', 'Fkbp1a', 
                                       'Ghr', 'Ifnar2', 'Ifngr1', 'Il11ra1', 'Il13ra1', 'Il1r1', 'Il1rl1', 'Il33', 
                                       'Il6', 'Il6st', 'Irak1', 'Itch', 'Jak1', 'Jak2', 'Kitl', 'Lepr', 'Lif', 'Lifr', 
                                       'Mif', 'Nars', 'Nucb2', 'Pdcl3', 'Pgf', 'Pik3r1', 'Rasl11b', 'Sdcbp', 'Siva1', 
                                       'Smad7', 'Snx6', 'Stat3', 'Tgfb1', 'Tgfb2', 'Tgfb3', 'Tgfbr3', 'Timm50', 'Tmbim1', 
                                       'Tnfsf12', 'Tnfsf9', 'Tollip', 'Trip6', 'Vegfa',]



cytokine_release = ['Csf1', 'Csf2', 'Ccl2', 'Cxcl1', 'Il11']




ECM_assembly_disassembly = ['Adamts15', 'Adamts5', 'Agt', 'Antxr1', 'Atp7a', 'Carmil2', 'Clasp1', 'Clasp2', 'Cma1', 'Col1a2', 'Col3a1', 'Ctsg', 'Ctss', 'Dag1', 'Ddr1', 'Ddr2', 'Dpp4', 'Efemp2', 
                            'Emilin1', 'Eng', 'Ets1', 'Exoc8', 'Fap', 'Fbln5', 'Fgfr4', 'Fkbp10', 'Flot1', 'Fscn1', 'Gpm6b', 'Hapln2', 'Has1', 'Has2', 'Has3', 'Hpn', 'Hpse', 'Ihh', 'Il6', 
                            'Kif9', 'Klk5', 'Lamb1', 'Lamb2', 'Lamb3', 'Lamc1', 'Lcp1', 'Lox', 'Ltbp3', 'Ltbp4', 'Meltf', 'Mfap4', 'Mmp13', 'Myh11', 'Notch1', 'Noxo1', 'Ntn4', 'Ntng1', 'Ntng2', 
                            'Pbxip1', 'Pdpn', 'Phldb1', 'Phldb2', 'Plod3', 'Pxdn', 'Qsox1', 'Ramp2', 'Rgcc', 'Sema5a', 'Sh3pxd2b', 'Smad3', 'Smpd3', 'Sox9', 'Tgfb1', 'Tgfb2', 'Thsd4', 
                            'Tie1', 'Tnxb'] # GO:0022617 + GO:0085029

ECM_assembly_disassembly_filtered = ['Agt', 'Antxr1', 'Col1a2', 'Col3a1', 'Dag1', 'Ddr2', 'Efemp2', 'Emilin1', 
                                     'Eng', 'Ets1', 'Fap', 'Fbln5', 'Fkbp10', 'Flot1', 'Fscn1', 'Gpm6b', 'Has1', 
                                     'Il6', 'Lamb1', 'Lamb2', 'Lamc1', 'Lox', 'Ltbp3', 'Ltbp4', 'Mfap4', 'Mmp13', 
                                     'Ntn4', 'Pbxip1', 'Pdpn', 'Phldb2', 'Pxdn', 'Qsox1', 'Rgcc', 'Sema5a', 'Sh3pxd2b', 
                                    'Smpd3', 'Tgfb1', 'Tgfb2', 'Tnxb' ]




hypoxia = ['Acaa2', 'Adam8', 'Ado', 'Adrb2', 'Ajuba', 'Ak4', 'Akt1', 'Aqp1', 'Aqp3', 'Atf2', 'Bad', 'Bcl2', 'Bmyc', 'Bnip3', 'Bnip3l', 'Cbs', 'Cd34', 'Cited2', 'Commd1', 'Cpeb1', 'Cpeb2', 'Cr1l', 
           'Cybb', 'Ddah1', 'Egln1', 'Egln2', 'Egln3', 'Egr1', 'Eif4ebp1', 'Eno1', 'Eno1b', 'Ep300', 'Epas1', 'Epha4', 'Fam162a', 'Fmn2', 'Fndc1', 'Gata6', 'Gnb1', 'Hif1a', 'Hif1an', 'Hif3a', 
           'Higd1a', 'Hp1bp3', 'Hyou1', 'Kcnk2', 'Kcnk3', 'Kdm6a', 'Map2k1', 'Mgarp', 'Mir214', 'Mlst8', 'Mtor', 'Myc', 'Ndnf', 'Nfe2l2', 'Nol3', 'Nop53', 'Notch1', 'Npepps', 'Ogt', 'P4hb', 
           'Pdk1', 'Pdk3', 'Pgk1', 'Pik3cb', 'Pink1', 'Plk3', 'Ppard', 'Pparg', 'Prkce', 'Pten', 'Ptgis', 'Rbpj', 'Rgcc', 'Rock2', 'Rora', 'Rptor', 'Rtn4', 'Rwdd3', 'Scn2a', 'Sdhd', 'Sirt1', 
           'Sirt2', 'Slc2a4', 'Slc8a3', 'Slc9a1', 'Stox1', 'Stub1', 'Suv39h1', 'Suv39h2', 'Tbl2', 'Tert', 'Tigar', 'Tmbim6', 'Trp53', 'Twist1', 'Ubqln1', 'Usp19', 'Vasn', 'Vegfa', 'Vhl', 'Vldlr', 
           'Zfas1', 'Zfp36l1']  # GO:0071456

hypoxia_filtered = ['Acaa2', 'Akt1', 'Aqp1', 'Bad', 'Bcl2', 'Bmyc', 'Bnip3', 'Bnip3l', 'Cd34', 'Commd1', 'Cr1l', 
                    'Egln1', 'Egr1', 'Eif4ebp1', 'Eno1', 'Ep300', 'Fam162a', 'Fndc1', 'Gnb1', 'Hif1a', 
                    'Higd1a', 'Hp1bp3', 'Myc', 'Nfe2l2', 'Nop53', 'Npepps', 'Ogt', 'P4hb', 'Pgk1', 'Pink1', 
                    'Ppard', 'Pten', 'Ptgis', 'Rbpj', 'Rgcc', 'Rora', 'Rtn4', 
                    'Sdhd', 'Sirt1', 'Sirt2', 'Slc9a1', 'Tmbim6', 'Trp53', 'Twist1', 'Ubqln1', 'Usp19', 'Vasn', 
                    'Vegfa', 'Vhl', 'Vldlr', 'Zfas1', 'Zfp36l1']




OXPHOS = ['1700066M21Rik', 'Abcd1', 'Actn3', 'Afg1l', 'Ak4', 'Atp5a1', 'Atp5b', 'Atp5c1', 'Atp5d', 'Atp5e', 'Atp5h', 'Atp5j', 'Atp5j2', 'Atp5k', 'Atp5o', 'Atp7a', 'Atpsckmt', 'Bdnf', 'Bid', 'Ccnb1', 
          'Cdk1', 'Chchd10', 'Coa6', 'Coq7', 'Coq9', 'Cox4i1', 'Cox4i2', 'Cox5a', 'Cox5b', 'Cox6a1', 'Cox6a2', 'Cox6c', 'Cox7a1', 'Cox7a2', 'Cox7a2l', 'Cox7c', 'Cox8a', 'Cox8b', 'Cyc1', 'Cycs', 
          'Dguok', 'Dld', 'Dnajc15', 'Dnajc30', 'Fxn', 'Iscu', 'Mlxipl', 'Msh2', 'mt-Atp6', 'mt-Atp8', 'mt-Co1', 'mt-Co2', 'mt-Co3', 'mt-Cytb', 'mt-Nd1', 'mt-Nd2', 'mt-Nd3', 'mt-Nd4', 'mt-Nd4l', 
          'mt-Nd5', 'mt-Nd6', 'Mtch2', 'Myc', 'Myog', 'Ndufa1', 'Ndufa2', 'Ndufa3', 'Ndufa5', 'Ndufa6', 'Ndufa7', 'Ndufa8', 'Ndufa9', 'Ndufa10', 'Ndufa11', 'Ndufa12', 'Ndufa13', 'Ndufab1', 'Ndufaf1',
          'Ndufb2', 'Ndufb3', 'Ndufb4', 'Ndufb5', 'Ndufb6', 'Ndufb7', 'Ndufb8', 'Ndufb9', 'Ndufb10', 'Ndufb11', 'Ndufc1', 'Ndufc2', 'Ndufs1', 'Ndufs2', 'Ndufs3', 'Ndufs4', 'Ndufs5', 'Ndufs6', 'Ndufs7', 
          'Ndufs8', 'Ndufv1', 'Ndufv2', 'Ndufv3', 'Nipsnap2', 'Nupr1', 'Park7', 'Pde2a', 'Pde12', 'Pink1', 'Ppif', 'Rhoa', 'Sdha', 'Sdhaf2', 'Sdhb', 'Sdhc', 'Sdhd', 'Shmt2', 'Slc25a23', 'Slc25a33', 
          'Snca', 'Stoml2', 'Tefm', 'Tnf', 'Uqcc2', 'Uqcc3', 'Uqcr10', 'Uqcr11', 'Uqcrb', 'Uqcrc1', 'Uqcrc2', 'Uqcrfs1', 'Uqcrh', 'Uqcrq', 'Vcp']  # GO:0006119

OXPHOS_filtered = ['Atp5a1', 'Atp5b', 'Atp5c1', 'Atp5d', 'Atp5e', 'Atp5h', 'Atp5j', 'Atp5j2', 'Atp5k', 
                   'Chchd10', 'Cox4i1', 'Cox4i2', 'Cox5a', 'Cox5b', 'Cox6a1', 'Cox6c', 'Cox7a2', 'Cox7a2l', 
                   'Cox7c', 'Cox8a', 'Cyc1', 'Dnajc15', 'Iscu', 'mt-Atp6', 'mt-Atp8', 'mt-Co1', 'mt-Cytb', 
                   'mt-Nd1', 'mt-Nd2', 'mt-Nd3', 'mt-Nd4', 'mt-Nd4l', 'mt-Nd5', 'mt-Nd6', 'Mtch2', 'Myc',
                   'Ndufa1', 'Ndufa2', 'Ndufa3', 'Ndufa5', 'Ndufa6', 'Ndufa7', 'Ndufa10', 'Ndufa11', 'Ndufa13', 
                   'Ndufb2', 'Ndufb3', 'Ndufb5', 'Ndufb7', 'Ndufb9', 'Ndufb10', 'Ndufb11', 'Ndufc2', 'Nupr1', 
                   'Park7', 'Rhoa', 'Sdhb', 'Sdhd', 'Shmt2', 'Stoml2', 'Uqcc2', 'Uqcr10', 'Uqcr11', 'Uqcrb', 'Uqcrq' ]



glycolysis = ['Actn3', 'Adpgk', 'Aldoa', 'Aldoart1', 'Aldob', 'Aldoc', 'App', 'Arl2', 'Bpgm', 'Cbfa2t3', 'Ddit4', 'Dhtkd1', 'Eif6', 'Eno1', 'Eno1b', 'Eno2', 'Eno3', 'Enpp1', 'Ep300', 'Fbp1', 'Flcn', 
              'Foxk1', 'Foxk2', 'Gale', 'Galk1', 'Galt', 'Gapdh', 'Gapdhs', 'Gck', 'Git1', 'Gpd1', 'Gpi1', 'Hdac4', 'Hif1a', 'Hk1', 'Hk2', 'Hk3', 'Hkdc1', 'Htr2a', 'Ier3', 'Igf1', 'Insr', 'Jmjd8', 
              'Khk', 'Mlst8', 'Mlxipl', 'Mpi', 'Mtch2', 'Mtor', 'Myc', 'Myog', 'Ncor1', 'Nupr1', 'Ogdh', 'Ogt', 'P2rx7', 'Pfkfb2', 'Pfkl', 'Pfkm', 'Pfkp', 'Pgam1', 'Pgk1', 'Pkm', 'Ppara', 'Ppargc1a', 
              'Prkaa1', 'Prkaa2', 'Prkag2', 'Prkag3', 'Prxl2c', 'Psen1', 'Rptor', 'Sirt6', 'Slc2a6', 'Slc4a1', 'Slc4a4', 'Stat3', 'Tigar', 'Tkfc', 'Tpi1', 'Zbtb7a', 'Zbtb20']  # GO:0006096


glycolysis_filtered = ['Aldoa', 'App', 'Arl2', 'Ddit4', 'Eif6', 'Eno1', 'Enpp1', 'Ep300', 'Flcn', 'Galk1', 
                       'Gapdh', 'Git1', 'Gpi1', 'Hdac4', 'Hif1a', 'Hk1', 'Ier3', 'Igf1', 'Mtch2', 'Myc', 'Ncor1', 
                       'Nupr1', 'Ogdh', 'Ogt', 'Pfkp', 'Pgam1', 'Pgk1', 'Pkm', 'Prxl2c', 'Stat3', 'Tpi1', 
                       'Zbtb7a', 'Zbtb20' ]



stress = ['Abcd1', 'Abl1', 'Adnp2', 'Agap3', 'Aif1', 'Aifm2', 'Akr1b3', 'Akr1c14', 'Akr1c18', 'Akr1cl', 'Akt1', 'Aldh2', 'Aldh3b1', 'Alox5', 'Ankzf1', 'Anxa1', 'Apex1', 'App', 'Aqp1', 'Arl6ip5', 
          'Arnt', 'Arntl', 'Atf2', 'Atf4', 'Atg7', 'Atp13a2', 'Atp2a2', 'Atp7a', 'Axl', 'Bag5', 'Bcl2', 'Bmp4', 'Bnip3', 'Brf2', 'Btk', 'Capn1', 'Cat', 'Ccs', 'Cd36', 'Cdkn2a', 'Cfl1', 'Chchd4', 
          'Chd6', 'Chuk', 'Coq7', 'Cpeb2', 'Ctnnb1', 'Cyp1b1', 'Dapk1', 'Ddias', 'Ddr2', 'Ddx3x', 'Dhfr', 'Diablo', 'Dnajc3', 'Ect2', 'Edn1', 'Ednra', 'Egfr', 'Eif2ak1', 'Eif2ak3', 'Eif2ak4', 
          'Eif2s1', 'Eif4g1', 'Endog', 'Epas1', 'Epor', 'Ercc6l2', 'Ermp1', 'Etv5', 'Ezh2', 'Fancc', 'Fancd2', 'Fbln5', 'Fbxo7', 'Fbxw7', 'Fech', 'Fer', 'Fos', 'Foxa1', 'Foxo1', 'Foxo3', 'Foxp1', 
          'Fut8', 'Fxn', 'Fyn', 'Fzd1', 'G6pdx', 'Gata4', 'Gch1', 'Gfer', 'Gpr37l1', 'Gpx1', 'Gskip', 'Gsr', 'Hdac2', 'Hdac6', 'Hgf', 'Hif1a', 'Hk3', 'Hmox1', 'Hsf1', 'Hspb1', 'Hsph1', 'Htra2', 
          'Il10', 'Il6', 'Impact', 'Itgam', 'Jak2', 'Jun', 'Kdm6b', 'Keap1', 'Krt13', 'Lancl1', 'Lcn2', 'Lonp1', 'Lrrk2', 'Map3k5', 'Mapk1', 'Mapk13', 'Mapk3', 'Mapk7', 'Mapk8', 'Mapk9', 'Mapkap1', 
          'Mapt', 'Mcl1', 'Meak7', 'Melk', 'Met', 'Mgat3', 'Mgst1', 'Mmp2', 'Mmp3', 'Mmp9', 'Mpo', 'Mpv17', 'Msra', 'Mt3', 'Nck1', 'Nck2', 'Ncoa7', 'Net1', 'Nfe2l1', 'Nfe2l2', 'Ngb', 'Ngfr', 
          'Nme5', 'Nol3', 'Nono', 'Nos3', 'Nox1', 'Npm1', 'Nqo1', 'Nr4a2', 'Nr4a3', 'Oser1', 'Oxr1', 'P4hb', 'Park7', 'Parp1', 'Pawr', 'Pcgf2', 'Pdcd10', 'Pde8a', 'Pdgfra', 'Pdgfrb', 'Pdk1', 'Pdk2', 
          'Pink1', 'Pjvk', 'Pla2r1', 'Plekha1', 'Pml', 'Pnpla8', 'Pnpt1', 'Ppargc1a', 'Ppargc1b', 'Ppia', 'Ppif', 'Ppp1r15a', 'Ppp1r15b', 'Prdx1', 'Prdx2', 'Prdx3', 'Prdx5', 'Prkaa1', 'Prkaa2', 
          'Prkcd', 'Prkch', 'Prkd1', 'Prkn', 'Prkra', 'Prr5l', 'Psap', 'Ptprk', 'Pycr1', 'Pycr2', 'Pyroxd1', 'Rack1', 'Rad52', 'Rbm11', 'Rbm4', 'Rela', 'Rest', 'Rhob', 'Ripk1', 'Ripk3', 'Rnf112', 
          'Rnf146', 'Rnf7', 'Romo1', 'Rps3', 'Rwdd1', 'Scly', 'Selenon', 'Selenos', 'Sesn2', 'Setx', 'Sfpq', 'Sigmar1', 'Sin3a', 'Sirpa', 'Sirt1', 'Sirt2', 'Slc11a2', 'Slc1a1', 'Slc25a24', 'Slc35a4', 
          'Slc4a11', 'Slc7a11', 'Slc8a1', 'Smpd3', 'Snca', 'Sod1', 'Sod2', 'Sod3', 'Sphk1', 'Src', 'Srxn1', 'Stat6', 'Stk24', 'Stk25', 'Stk26', 'Stx2', 'Stx4a', 'Tbc1d24', 'Tlr4', 'Tlr6', 'Tmed2', 
          'Tmem161a', 'Tnf', 'Tnfaip3', 'Trap1', 'Trem2', 'Trp53', 'Trp53inp1', 'Trpc6', 'Trpm2', 'Tsc1', 'Txn1', 'Txndc2', 'Uaca', 'Ubqln1', 'Ucp1', 'Vkorc1l1', 'Vnn1', 'Vrk2', 'Wnt1', 'Wnt16', 
          'Xbp1', 'Zc3h12a', 'Zfp277', 'Zfp580', 'Zfp622']  # GO:0043555 + GO:1900407


stress_filtered = ['Abl1', 'Akt1', 'Aldh2', 'Anxa1', 'App', 'Aqp1', 'Arl6ip5', 'Arntl', 'Atf4', 'Atp2a2', 'Axl', 
                   'Bag5', 'Bcl2', 'Bmp4', 'Bnip3', 'Cfl1', 'Ctnnb1', 'Ddr2', 'Ddx3x', 'Ednra', 'Fbln5', 
                   'Fos', 'Foxo3', 'Foxp1', 'Fyn', 'Fzd1', 'Gpx1', 'Gsr', 'Hif1a', 'Hmox1', 'Hspb1', 'Hsph1', 
                   'Jak2', 'Jun', 'Kdm6b', 'Lcn2', 'Mapk3', 'Mapkap1', 'Mcl1', 'Mgst1', 'Mmp2', 'Mmp3', 
                   'Nfe2l1', 'Nfe2l2', 'Nono', 'Nr4a2', 'P4hb', 'Park7', 'Pdcd10', 'Pdgfra', 'Pdgfrb', 'Ppia', 
                   'Ppp1r15a', 'Prdx1', 'Prdx2', 'Prdx5', 'Psap', 'Rack1', 'Rela', 'Rhob', 'Romo1', 'Rwdd1', 
                   'Selenos', 'Sfpq', 'Sirpa', 'Smpd3', 'Sod1', 'Sod2', 'Srxn1', 'Stx4a', 'Tmed2', 'Tnfaip3', 
                   'Trp53', 'Txn1', 'Uaca', 'Xbp1', 'Zfp622', ]


positive_regulation_of_acute_inflammatory_response = ['Adam8', 'Alox5ap', 'Aoc3', 'Btk', 'C2cd4a', 'C2cd4b', 'C3', 'Ccl5', 'Ccr7', 
                        'Cnr1', 'Fcer1a', 'Fcer1g', 'Fcgr1', 'Fcgr3', 'Ffar2', 'Ffar3', 
                        'Ighg1', 'Ighg2b', 'Il1b', 'Il6', 'Npy5r', 'Park7', 'Pik3cg', 
                        'Ptger3', 'Ptgs2', 'Tac1', 'Tnf', 'Tnfrsf11a', 'Tnfsf11', 'Zp3', ]  # GO:0002675

positive_regulation_of_acute_inflammatory_response_filtered = [
                         'Alox5ap', 'Aoc3', 'C3', 'Ccl5', 'Il6', 'Park7', 
                        'Ptger3', 'Ptgs2', 'Tac1', 'Tnfsf11', 
                        ]



positive_regulation_of_inflammatory_response = ['Abcc1', 'Ace', 'Adam8', 'Adora2b', 'Adora3', 'Ager', 'Aim2', 'Alox5ap', 'Aoc3', 
                                                'App', 'Btk', 'C2cd4a', 'C2cd4b', 'C3', 'Camk2n1', 'Casp1', 'Casp4', 'Ccl1', 'Ccl24', 
                                                'Ccl3', 'Ccl5', 'Ccn4', 'Ccr2', 'Ccr5', 'Ccr7', 'Cd24a', 'Cd28', 'Cd47', 'Cd81', 
                                                'Cdk19', 'Cebpa', 'Cebpb', 'Clock', 'Cnr1', 'Csf1r', 'Ctsc', 'Ctss', 'Ddt', 'Dhx9', 
                                                'Ednra', 'Egfr', 'Ets1', 'Fabp4', 'Fcer1a', 'Fcer1g', 'Fcgr1', 'Fcgr3', 'Fem1a', 
                                                'Ffar2', 'Ffar3', 'Fgfr1', 'Gpr4', 'Gprc5b', 'Gpsm3', 'Grn', 'Hspd1', 'Hyal2', 'Ido1', 
                                                'Ifi35', 'Ifng', 'Ighg1', 'Ighg2b', 'Il16', 'Il17ra', 'Il17rb', 'Il18', 
                                                'Il1b', 'Il1rl1', 'Il33', 'Il6', 'Itga2', 'Jak2', 'Kars', 'Lbp', 'Ldlr', 'Lgals1', 
                                                'Lgals2', 'Lilra5', 'Lpl', 'Lrrk2', 'Lta', 'Mdk', 'Mefv', 'Mmp8', 'Napepld', 'Nfkbia', 
                                                'Nfkbiz', 'Ninj1', 'Nkg7', 'Nlrp10', 'Nlrp1a', 'Nlrp1b', 'Nlrp3', 'Nlrp6', 'Nmi', 
                                                'Nppa', 'Npy5r', 'Nupr1', 'Osm', 'Park7', 'Pdcd4', 'Pde2a', 'Pde5a', 'Pik3cg', 
                                                'Pla2g3', 'Pla2g4a', 'Plcg2', 'Prkca', 'Ptger3', 'Ptger4', 'Ptgs2', 'Pycard', 
                                                'Ripk1', 'Rps19', 'S100a8', 'S100a9', 'Serpine1', 'Setd4', 'Snca', 'Snx4', 'Stap1', 
                                                'Stat5a', 'Stat5b', 'Sucnr1', 'Tac1', 'Tafa3', 'Tgm2', 'Tlr2', 'Tlr3', 'Tlr4', 'Tlr6', 
                                                'Tlr9', 'Tnf', 'Tnfrsf11a', 'Tnfrsf1a', 'Tnfsf11', 'Tnfsf18', 'Tnfsf4', 'Tnip1', 
                                                'Tradd', 'Trem2', 'Trpv4', 'Tslp', 'Ttbk1', 'Vamp7', 'Vamp8', 'Wnt5a', 'Zbp1', 'Zp3']
  #  GO:0050729 

positive_regulation_of_inflammatory_response_filtered = [
                        'Abcc1', 'Ace','App',  'C3',  'Casp4', 'Ccl5', 'Ccn4', 'Cd24a', 'Cd47', 'Cd81', 
                        'Cebpb', 'Ctsc', 'Ddt', 'Dhx9', 'Egfr', 'Ets1', 'Fabp4', 'Fgfr1', 'Grn', 'Hspd1', 
                        'Ifi35', 'Il17ra', 'Il1rl1', 'Il33', 'Il6', 'Jak2', 'Lbp', 'Lgals1', 
                        'Lpl', 'Mdk', 'Nfkbia', 'Nfkbiz', 'Ninj1', 'Nupr1', 'Park7', 'Pdcd4', 'Pla2g4a', 'Prkca', 
                        'Ptgs2', 'Tac1', 'Tgm2', 'Tlr2', 'Tnfrsf1a', 
                        'Tnfsf11', 'Tnip1', 'Trpv4', 'Vamp8', 'Wnt5a', 
                        ]




positive_regulation_of_lipid_biosynthetic_process = [
    'Abcd1','Abcd2', 'Abcg1', 'Abcg4', 'Abhd6', 'Acsl3', 'Acsl5', 'Adgrf5', 'Adora2b', 'Agt', 'Akt1', 'Anxa1', 'Apoa1',
    'Apoe', 'Avp', 'Avpr1a', 'Bmp6', 'Capn2', 'Ccdc3', 'Ccn1', 'Cd74',
    'Chp1', 'Cnep1r1', 'Creb1', 'Crebl2', 'Ctdnep1', 'Cyp17a1', 'Dab2', 'Dbi', 'Dgat1', 'Dgat2', 'Elovl5', 
    'Fabp3', 'Fdps', 'Fgf1', 'Gpam', 'Gpld1', 'Hsd17b13', 'Htr2a', 'Htr2b', 
    'Igf1', 'Igf1r', 'Igf2', 'Il1b', 'Kat5', 'Ldlr', 'Lpgat1', 'Mapk9', 'Mfsd2a', 'Mid1ip1', 'Mlst8', 'Mlxipl', 
    'Mtor', 'Nr1d1', 'Nr1h2', 'Nr1h3', 'Pcx', 'Pla2g3', 'Pla2g4a', 'Pla2g6', 'Plin5', 'Por', 'Ppara', 'Ppargc1a', 
    'Prkcd', 'Ptgs2', 'Rab38', 'Rdh1', 'Rdh10', 'Rdh16', 'Rdh9', 'Rptor', 'Scap', 'Scarb1', 'Scp2', 
    'Sec14l2', 'Sirt3', 'Sirt4', 'Slc27a1', 'Slc45a3', 'Smpd3', 'Sorbs1', 'Sphk2', 'Srebf1', 'Srebf2', 'Star', 'Stard4', 
    'Tcf7l2', 'Tnf', 'Wnt4', 'Zbtb20'] # GO:0046889



positive_regulation_of_lipid_biosynthetic_process_filtered = [
    'Acsl3', 'Agt', 'Akt1', 'Anxa1', 'Apoe', 'Bmp6', 'Capn2', 'Ccdc3', 'Ccn1', 
    'Chp1', 'Cnep1r1', 'Ctdnep1', 'Dab2', 'Dbi', 'Igf1', 'Igf1r', 'Igf2', 'Lpgat1', 'Mid1ip1',  
    'Nr1d1', 'Nr1h2', 'Pla2g4a', 'Por', 'Prkcd', 'Ptgs2', 'Rdh10', 'Rptor', 'Scap', 'Scarb1', 'Scp2', 
    'Slc27a1', 'Sorbs1', 'Srebf2', 'Tcf7l2', 'Zbtb20'] # GO:0046889

In [ ]:
fb_order = list(dict_fbs.keys())
fb_num_order = [i.split(': ')[0] for i in fb_order]

cmap='Blues'

In [ ]:
adata_fb.obs['condition-cell_type'] = adata_fb.obs['subtype'].astype(str) + '-' + adata_fb.obs['condition'].astype(str)
adata_fb.obs['condition-cell_type'] = adata_fb.obs['condition-cell_type'].astype('category')
adata_fb.obs['condition-cell_type'].cat.categories

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=positive_regulation_of_lipid_biosynthetic_process_filtered, 
                 use_raw=False, categories_order=fb_order, cmap=cmap, show=False, 
                 dendrogram=False, swap_axes=True, figsize=(10, 8), vmax=0.7)

plt.savefig(f'results/figures/F2/matrixplot_positive_regulation_of_lipid_biosynthetic_process_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=positive_regulation_of_inflammatory_response_filtered, use_raw=False,
                 dendrogram=False, swap_axes=True, figsize=(10, 14), vmax=1, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_positive_regulation_of_inflammatory_response_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=positive_regulation_of_acute_inflammatory_response_filtered, 
                 dendrogram=False, swap_axes=True, figsize=(10, 4), vmax=0.85, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_positive_regulation_of_acute_inflammatory_response_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=complement_genes_filtered, dendrogram=False, 
                 swap_axes=True, figsize=(10, 4), vmax=0.85, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_complement_genes_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=cytokine_interaction_genes_filtered, dendrogram=False, 
                 swap_axes=True, figsize=(10, 13), vmax=0.85, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_cytokine_interaction_genes_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=cytokine_release, dendrogram=False, swap_axes=True, 
                 figsize=(10, 1), vmax=0.85, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_cytokine_release_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=ECM_assembly_disassembly_filtered, dendrogram=False, 
                 swap_axes=True, figsize=(10, 6), vmax=0.85, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_ECM_assembly_disassembly_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=hypoxia_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 8), vmax=0.85, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_hypoxia_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=OXPHOS_filtered, dendrogram=False, swap_axes=True, figsize=(10, 10), 
                 vmax=1.6, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_OXPHOS_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=glycolysis_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 5), vmax=0.85, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_glycolysis_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='subtype', var_names=stress_filtered, dendrogram=False, swap_axes=True, 
                 figsize=(10, 11), vmax=1.2, categories_order=fb_order, cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_stress_filtered_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

## UMAP of scores of selected pathways

In [ ]:
path_names = ['Complement', 'Cytokine', 'ECM', 'Hypoxia', 'OXPHOS', 'Glycolysis', 'Stress', 
                       'Inflammatory response', 'Acute inflammatory', 'Lipid biosynthesis']
path_lists = [complement_genes, cytokine_interaction_genes, ECM_assembly_disassembly, 
                       hypoxia, OXPHOS, glycolysis, stress, positive_regulation_of_inflammatory_response, 
                       positive_regulation_of_acute_inflammatory_response, 
                       positive_regulation_of_lipid_biosynthetic_process]

for path_name, path_list  in zip(path_names, path_lists):
    sc.tl.score_genes(adata_fb, gene_list=path_list, ctrl_size=50, gene_pool=None, n_bins=25, \
                      score_name=f'{path_name}', random_state=0, copy=False, use_raw=False)

    
sc.pl.umap(adata_fb, color=['subtype_number'] + path_names, 
           ncols=3, frameon=False, cmap=cmap, legend_loc = 'on data', show=False, 
           legend_fontoutline=4, legend_fontsize=14,  vmin="p15",  vmax="p95")

plt.savefig(f'results/figures/F2/UMAP_path_scores_fb_{cmap}_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

In [ ]:
selected_paths = ['Hypoxia', 'Glycolysis', 'Stress', 'Cytokine']

fig, axs = plt.subplots(2, 2, figsize=(9, 6))

for idx, path in enumerate(selected_paths):
    sc.pl.umap(adata_fb, color=path, ax=axs.ravel()[idx],
               ncols=2, frameon=False, cmap=cmap, legend_loc = 'on data', show=False, 
               legend_fontoutline=4, legend_fontsize=14,  vmin="p15",  vmax="p95")
    
    axs.ravel()[idx].title.set_size('xx-large')

plt.savefig(f'results/figures/F2/UMAP_path_scores_selected_fb_{cmap}_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)
plt.savefig(f'results/figures/F2/Fig2B.jpg', dpi=300)

In [ ]:
n_cols = 3
n_rows = int(len(path_names) / n_cols) + (len(path_names) % n_cols > 0)

fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 2.5))

for idx, gene in enumerate(path_names):
    sc.pl.violin(adata_fb, keys=gene, groupby='subtype_number', rotation=90, ax = axs.ravel()[idx], show=False, 
                 order=fb_num_order)
    
for idx in range(len(path_names), n_rows*n_cols):
    axs.ravel()[idx].set_axis_off()

plt.tight_layout()

plt.savefig(f'results/figures/F2/violinplot_path_scores_fb_{cmap}_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

## UMAP/heatmap highlighting the expression of specific markers

In [ ]:
markers = {'Carbohydrate metabolism': ['Slc2a1', 'Slc2a3', 'Pfkfb3', 'Aldoa', 'Gapdh', 'Pgam1', 'Eno1', 'Pgk1', 
                                       'Ldha', 'Slc16a4', 'Pdk1'],  # Pkm2 not present
           'Hypoxia-induced autophagy and stress response': ['Hif1a', 'Bnip3', 'Bnip3l', 'Malat1', 'Neat1', 'Ero1l', 
                                                             'Ier3', 'Mif'],
           'Lipid metabolism': ['Vldlr', 'Lrp1', 'Fabp5', 'Plin2', 'Hilpda',],
           'Aminoacid transport and metabolism': ['Slc3a2', 'Slc7a2', 'Slc7a1', 'Nos2', 'Arg1', 'Arg2'], 
           'Angiogenesis': ['Vegfa', 'Vegfb', 'Vegfc', 'Pgf', ], 
           'Proinflammatory cytokines': ['Il1b', 'Tnf', 'C3', 'Cxcl1', 'Cxcl2', 'Cxcl3', 
                                         'Apobec1', 'Apobec3'], # 'Il1a' not present
           'CAF': ['Pdpn', 'Pdgfra',  'Dcn', 'Il6', 'Cxcl1', 'Cxcl2', 'Ccl2', 
                   'Cxcl12', 'Has1', 'Has2', 'Cd44', 'Il1r1'] # Cd24 not present
          }


sc.pl.matrixplot(adata_fb, var_names=markers, groupby='condition-cell_type', dendrogram=False, 
                 use_raw=False, swap_axes=True,  vmax=0.8, figsize=(7, 12), cmap=cmap, show=False)

plt.savefig(f'results/figures/F2/matrixplot_selected_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)

## UMAPs of selected markers

In [ ]:
def plot_WT_KO_genes(adata, genes, n_cols=3, figsize=None, plot_labels_batch=True, plot_KO_vs_WT=True):
    p_p = plot_labels_batch + plot_KO_vs_WT
    
    adata_WT, adata_KO = adata[adata.obs['batch'].isin(['WT1', 'WT2'])], adata[adata.obs['batch'].isin(['KO1', 'KO2'])]
    adata_WT.uns['batch_colors'] = [dict_WT_KO_colors['WT1'], dict_WT_KO_colors['WT2']]
    adata_KO.uns['batch_colors'] = [dict_WT_KO_colors['KO1'], dict_WT_KO_colors['KO2']]
    
    n_rows = int((len(genes) + p_p) / n_cols) + ((len(genes) + p_p) % n_cols > 0)
    
    if figsize is None:
        figsize = (n_cols * 4, n_rows * 2 * 2.5)
    
    fig, axs = plt.subplots(n_rows * 2, n_cols, figsize=figsize)
    
    start = 0
    if plot_labels_batch:
        sc.pl.umap(adata, color='batch', frameon=False, cmap=magma, title='batch',
                   show=False, ax=axs[1][start])
        sc.pl.umap(adata, color='subtype_number', frameon=False, cmap=magma, show=False, title='Subtype',
                   legend_fontoutline=4, legend_fontsize='x-large', legend_loc='on data', ax=axs[0][start])
        start += 1
        
    if plot_KO_vs_WT:
        sc.pl.umap(adata_WT, color='batch', frameon=False, cmap=magma, title='WT',
                   show=False, ax=axs[0][start])
        sc.pl.umap(adata_KO, color='batch', frameon=False, cmap=magma, title='KO',
                   show=False, ax=axs[1][start])
    
    for idx, gene in enumerate(genes, start=p_p):
        row_gene = int(idx / n_cols) 
        col_gene = idx % n_cols
        
        min_gene, max_gene = adata[:, gene].X.min(), adata[:, gene].X.max()
        
        sc.pl.umap(adata_WT, color=gene, frameon=True, cmap=magma, use_raw=False,
                   show=False, legend_loc='on data', ax=axs[row_gene * 2][col_gene], 
                   vmin=min_gene, vmax=max_gene)
        
        sc.pl.umap(adata_KO, color=gene, frameon=True, cmap=magma, use_raw=False,
                   show=False, legend_loc='on data', ax=axs[row_gene * 2 + 1][col_gene], 
                   vmin=min_gene, vmax=max_gene, title='')        
    
    
    for row in range(n_rows):
        axs[2 * row][0].set_ylabel('WT', fontsize=pylab.rcParams['font.size'] * 2.5)
        axs[2 * row + 1][0].set_ylabel('KO', fontsize=pylab.rcParams['font.size'] * 2.5)
    
    for ax in axs[::2, :].ravel():
        ax.title.set_fontsize(pylab.rcParams['font.size'] * 2.5)
    
    for ax in axs[:, 1:].ravel():
        ax.set_ylabel('')
    
    for ax in axs.ravel():
        ax.set_xlabel('')
        for spine in ax.spines.values():
            spine.set_visible(False)
        
    # clean axes
    for col in range(col_gene + 1, n_cols):
        axs[row_gene * 2][col].set_axis_off()
        axs[row_gene * 2 + 1][col].set_axis_off()
    
    plt.tight_layout()

In [ ]:
list_markers_up = ['Il1rl1', 'Pdpn', 'C3', 'Saa3', 'Acta2', 'Tagln'] 
list_markers_down = ['Il1r1','Tnfrsf1b', 'Ifngr1', 'Il6ra', 'Tgfbr3', 'Vegfc']

plot_WT_KO_genes(adata_fb, genes=list_markers_up + list_markers_down, n_cols=6, 
                     figsize=None, plot_labels_batch=False, plot_KO_vs_WT=False)

plt.savefig(f'results/figures/F2/UMAP_selected_markers_{date.today().strftime("%Y_%m_%d")}.jpg', dpi=300)
plt.savefig(f'results/figures/F2/Fig2C.jpg', dpi=300)

In [ ]:
list_markers_up = ['Entpd1', 'Il11'] 

plot_WT_KO_genes(adata_fb, genes=list_markers_up , n_cols=3, 
                     figsize=None, plot_labels_batch=True, plot_KO_vs_WT=False)


### Save adata

In [ ]:
adata_fb.write_h5ad('adatas/adata_fibroblast.h5')